# Example: solute-grain boundary binding

This saved workflow demonstrates how solute-grain boundary (GB) binding can be calculated in ironflow.

In this case, we'll be examining the per-solute-atom energy cost of moving solutes from the bulk into a particular decoration pattern at the GB. I.e.

$E_\mathrm{bind}^{X}$ = (E_\mathrm{GB}^X + \eta E_\mathrm{bulk}) - (E_\mathrm{GB} + \eta E_\mathrm{bulk}^X)$

Where $X$ denotes the presence of the solute species, bulk and GB denote the structure, and $\eta$ is the ratio of solute atoms found in the GB structure (2 in the saved workflow below) and the bulk structure (1 in the saved workflow below).

In [ ]:
%matplotlib inline
from ironflow import GUI, node_tools as nt
from pyiron_atomistics import Atoms

We have most of the tools we need to do this calculation already in ironflow, but we'll need to add one final node to take our simulation results and do the math described above.

We'll define that node here in the notebook, then register it with ironflow when we instantiate the GUI.

In [ ]:
class BindingEnergy_Node(nt.DataNode):
    """
    The average (per-solute) energy associated with moving $n$ solutes from the bulk 
    into a particular decoration pattern at a grain boundary.
    """
    
    title = "BindingEnergy"
    init_inputs = [
        nt.NodeInputBP(label="bulk_en", dtype=nt.dtypes.Float()),
        nt.NodeInputBP(label="bulk_x_en", dtype=nt.dtypes.Float()),
        nt.NodeInputBP(label="gb_en", dtype=nt.dtypes.Float()),
        nt.NodeInputBP(label="gb_x_en", dtype=nt.dtypes.Float()),
        nt.NodeInputBP(
            label="bulk_x_structure",
            dtype=nt.dtypes.Data(valid_classes=[Atoms])
        ),
        nt.NodeInputBP(
            label="gb_x_structure", 
            dtype=nt.dtypes.Data(valid_classes=[Atoms])
        ),
        nt.NodeInputBP(label="solute_species", dtype=nt.dtypes.String(default=None)),
    ]
    init_outputs = [
        nt.NodeOutputBP(label="binding_energy", dtype=nt.dtypes.Float())
    ]
    
    def node_function(
        self, 
        bulk_en, 
        bulk_x_en, 
        gb_en, gb_x_en, 
        bulk_x_structure, 
        gb_x_structure, 
        solute_species
    ):
        n_solutes_in_bulk = (bulk_x_structure.get_chemical_symbols() == solute_species).sum()
        n_solutes_at_gb = (gb_x_structure.get_chemical_symbols() == solute_species).sum()
        bulks_per_gb = n_solutes_at_gb / n_solutes_in_bulk
        
        segregated_energy = gb_x_en + bulks_per_gb * bulk_en
        clean_energy = gb_en + bulks_per_gb * bulk_x_en
        return {"binding_energy": (segregated_energy - clean_energy) / n_solutes_at_gb}

In [ ]:
gui = GUI('solute_gb_binding', extra_nodes_packages=[[BindingEnergy_Node]])

The workflow graph is all ready to go; calculation nodes that run an underyling pyiron job are "slow", i.e., unlike the "fast" `DataNode` nodes, they need to explicitly receive a `run` command to produce output. Here we've chained them together by connecting their `ran` output signal port to the next calculations `run` input port -- so just hit `run` on the pure bulk structure and the rest will go!

Zoom out with the negative magnifying glass button twice to see the whole workflow at once.

In [ ]:
gui.draw()

If you want to modify the calculation, e.g. to change the solute species, don't forget to `reset` each of the minimization nodes.

As an exercise, you may wish to change the host species, or the GB being looked at -- perhaps changing the GB character, or maybe inserting a `Repeat` node before adding solutes to look at the dilute segregation energy. This is possible, but note that the decoration pattern is defined by the `InputArray` node used to define the `indices` for species change -- if you make any changes to the GB in question, you'll need to examine it (click `SHOW` and look at the `plot3d`) and choose new indices for your desired decoration pattern (clicking on individual atoms in the NGLView presentation of the structure shows their index, sometimes with a `^` character breaking up the number, i.e. `4^2` is `42`).

Note that the `Lammps` node normally automatically populates its `potential` input based on its `structure`. In this case, we want to make sure that both the bulk and solute-containing calculations use the _same potential_ in order to get equivalent representations of the host species. To accomplish this, we just manually select a potential from the `LammpsPotentials` node after giving it one of the _solute containing_ structures.